<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/Recursive_SQL_with_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-03-01 10:25:32.176148+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#Install Postgres in VM

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
!sudo -u postgres psql -V
!service postgresql start





psql (PostgreSQL) 14.11 (Ubuntu 14.11-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


## Configure

In [3]:
# Setup a password `pass` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
#
# Setup a database with name `praxisdb` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS praxisdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE praxisdb;'
#
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:praxisdb:postgres:pass" > ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

ALTER ROLE
NOTICE:  database "praxisdb" does not exist, skipping
DROP DATABASE
CREATE DATABASE
localhost:5432:praxisdb:postgres:pass


#ChatGPT Prompt

##ChatGPT prompt<br>
Can  you please help me with writing a recursive SQL for postgreSQL database<br>



###Reply
Certainly! Recursive SQL in PostgreSQL is often used for querying hierarchical data stored in a table with a self-referencing foreign key. Here's an example of how you can write a recursive SQL query in PostgreSQL:

Let's suppose you have a table called employees with columns employee_id and manager_id, where manager_id refers to the employee_id of the manager of each employee.
WITH RECURSIVE EmployeeHierarchy AS ( SELECT employee_id, manager_id, 1 AS level FROM employees WHERE manager_id IS NULL -- This selects the top-level employees

UNION ALL

SELECT e.employee_id, e.manager_id, eh.level + 1 FROM employees e JOIN EmployeeHierarchy eh ON e.manager_id = eh.employee_id ) SELECT * FROM EmployeeHierarchy ORDER BY level, employee_id;
In this recursive query:

EmployeeHierarchy is the name given to the recursive query. The initial part before UNION ALL selects the top-level employees (those who don't have a manager, indicated by manager_id IS NULL). The recursive part (UNION ALL onward) selects employees who are managed by the employees already in the EmployeeHierarchy set, and it keeps repeating until no more records match the recursive part. level is used to keep track of the hierarchy depth. You can customize this query based on your specific table structure and requirements. Make sure to replace employees with the actual name of your table, and adjust column names accordingly.

## Table Set Up

In [6]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c \
"create table e1 \
(e_id char(6),\
 m_id char(20)\
);\
"

CREATE TABLE


In [12]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "truncate table e1"

TRUNCATE TABLE


In [14]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c \
"insert into e1 (e_id, m_id) values \
('e001',NULL),\
('e002','e001'),\
('e003','e001'),\
('e004','e002'),\
('e005','e002'),\
('e006','e002'),\
('e007','e004'),\
('e008','e004'),\
('e009','e005'),\
('e010','e005')\
;\
"

INSERT 0 10


In [15]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "select * from e1"

  e_id  |         m_id         
--------+----------------------
 e001   | 
 e002   | e001                
 e003   | e001                
 e004   | e002                
 e005   | e002                
 e006   | e002                
 e007   | e004                
 e008   | e004                
 e009   | e005                
 e010   | e005                
(10 rows)



In [17]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
WITH RECURSIVE EmployeeHierarchy AS ( SELECT e_id, m_id, 1 AS level FROM e1 WHERE m_id IS NULL \
UNION ALL \
SELECT e.e_id, e.m_id, eh.level + 1 FROM e1 e JOIN EmployeeHierarchy eh ON e.m_id = eh.e_id ) \
SELECT * FROM EmployeeHierarchy ORDER BY level, e_id;\
"

  e_id  |         m_id         | level 
--------+----------------------+-------
 e001   |                      |     1
 e002   | e001                 |     2
 e003   | e001                 |     2
 e004   | e002                 |     3
 e005   | e002                 |     3
 e006   | e002                 |     3
 e007   | e004                 |     4
 e008   | e004                 |     4
 e009   | e005                 |     4
 e010   | e005                 |     4
(10 rows)



##ChatGPT Prompt
thanks, Now I want to reverse the query so that given a employee, I want to find all the upline managers. Please modify the query accordingly

###Reply
To reverse the query and find all the upline managers given an employee, you can modify the recursive part of the query to traverse upwards through the managerial hierarchy. Here's the modified query:<br>
WITH RECURSIVE ManagerialHierarchy AS (
  SELECT employee_id, manager_id, 1 AS level
  FROM employees
  WHERE employee_id = :input_employee_id -- Replace :input_employee_id with the employee ID you're interested in

  UNION ALL

  SELECT e.employee_id, e.manager_id, mh.level + 1
  FROM employees e
  JOIN ManagerialHierarchy mh ON e.employee_id = mh.manager_id
)
SELECT *
FROM ManagerialHierarchy
ORDER BY level DESC; -- Ordering by level in descending order will give you the hierarchy from the employee up to the top-level manager.
<br>
In this modified query:

ManagerialHierarchy is the name given to the recursive query.
The initial part before UNION ALL selects the employee you're interested in by specifying its employee_id.
The recursive part (UNION ALL onward) selects managers of the employee found in the previous iteration, effectively moving upwards in the hierarchy.
level is used to keep track of the hierarchy depth.
Finally, we order the result set by level in descending order to get the hierarchy from the given employee up to the top-level manager.
Make sure to replace employees with the actual name of your table, and adjust column names accordingly. Also, replace :input_employee_id with the specific employee ID you want to start the hierarchy traversal from.


In [18]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
WITH RECURSIVE ManagerialHierarchy AS ( \
  SELECT e_id, m_id, 1 AS level \
  FROM e1 \
  WHERE e_id = 'e009'  \
  UNION ALL \
  SELECT e.e_id, e.m_id, mh.level + 1 \
  FROM e1 e \
  JOIN ManagerialHierarchy mh ON e.e_id = mh.m_id \
) \
SELECT * \
FROM ManagerialHierarchy \
ORDER BY level DESC; \
"

  e_id  |         m_id         | level 
--------+----------------------+-------
 e001   |                      |     4
 e002   | e001                 |     3
 e005   | e002                 |     2
 e009   | e005                 |     1
(4 rows)



In [19]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
WITH RECURSIVE ManagerialHierarchy AS ( \
  SELECT e_id, m_id, 1 AS level \
  FROM e1 \
  WHERE e_id = 'e009'  \
  UNION ALL \
  SELECT e.e_id, e.m_id, mh.level + 1 \
  FROM e1 e \
  JOIN ManagerialHierarchy mh ON e.e_id = mh.m_id \
) \
SELECT * \
FROM ManagerialHierarchy \
ORDER BY level ; \
"

  e_id  |         m_id         | level 
--------+----------------------+-------
 e009   | e005                 |     1
 e005   | e002                 |     2
 e002   | e001                 |     3
 e001   |                      |     4
(4 rows)



#ChatGPT does even more

## Prompt
I need a PostgreSQL table which should have the following columns, memberid char(5) primary key, member name varchar(10), recommendedby char(5) foreign key referring to memberid that shows which member referred this member. Can you please give me the create table statement

### Reply
Certainly! Here's the CREATE TABLE statement for your PostgreSQL table: <br>
CREATE TABLE members (
    memberid CHAR(5) PRIMARY KEY,
    member_name VARCHAR(10),
    recommendedby CHAR(5),
    FOREIGN KEY (recommendedby) REFERENCES members(memberid)
);
<br>
n this table:

memberid is the primary key column, ensuring each member has a unique identifier.
member_name is a VARCHAR column to store the name of the member.
recommendedby is a CHAR(5) column that stores the memberid of the member who referred this member. It's also a foreign key referencing the memberid column of the same table, ensuring referential integrity.
This CREATE TABLE statement should create the table you described.

In [21]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
CREATE TABLE members \
( \
memberid CHAR(5) PRIMARY KEY, \
member_name VARCHAR(10), \
recommendedby CHAR(5), FOREIGN KEY (recommendedby) REFERENCES members(memberid) \
)\
;\
"

CREATE TABLE


##Prompt
For the members table described above, please create 12 pieces of sample data with Indian names and give me the insert statements for the same

In [22]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
INSERT INTO members (memberid, member_name, recommendedby) VALUES \
('M0001', 'Ravi', NULL), \
('M0002', 'Priya', 'M0001'), \
('M0003', 'Sanjay', 'M0001'), \
('M0004', 'Neha', 'M0002'), \
('M0005', 'Amit', 'M0002'), \
('M0006', 'Anjali', 'M0003'), \
('M0007', 'Suresh', 'M0003'), \
('M0008', 'Meena', 'M0003'), \
('M0009', 'Rahul', 'M0005'), \
('M0010', 'Kavita', 'M0006'), \
('M0011', 'Vivek', 'M0007'), \
('M0012', 'Sunita', 'M0008'); \
"

INSERT 0 12


## Prompt
Give me SQL that will show the chain of members recommended by 'Priya' and the people that she recommended

### Reply
To retrieve the chain of members recommended by 'Priya' and the people that she recommended, you can use a recursive common table expression (CTE) to traverse the hierarchy. Here's the SQL query for that: <br>
This query:

Uses a recursive CTE named MemberChain.
The initial part selects members recommended by 'Priya' (M0002). You should replace 'M0002' with the memberid of 'Priya'.
The recursive part joins the members table with the MemberChain CTE to find the members recommended by the previously selected members.
It keeps traversing the hierarchy until no more records match the recursive part.
Finally, it selects all the columns from the MemberChain CTE and orders the results by the level of recommendation.
This query will give you a chain of members recommended by 'Priya' and the members she directly recommended, along with their recommendation levels.

In [24]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
WITH RECURSIVE MemberChain AS ( \
  SELECT memberid, member_name, recommendedby, 1 AS level \
  FROM members \
  WHERE recommendedby = 'M0002'  \
  UNION ALL \
  SELECT m.memberid, m.member_name, m.recommendedby, mc.level + 1 \
  FROM members m \
  JOIN MemberChain mc ON m.recommendedby = mc.memberid \
) \
SELECT * \
FROM MemberChain \
ORDER BY level;\
"

 memberid | member_name | recommendedby | level 
----------+-------------+---------------+-------
 M0004    | Neha        | M0002         |     1
 M0005    | Amit        | M0002         |     1
 M0009    | Rahul       | M0005         |     2
(3 rows)



## Prompt
Please modify this query for an upward chain, which means, given a member name, I would like to know who he was recommended by and who in turn recommended the recommender and so on

In [27]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
WITH RECURSIVE MemberChain AS ( \
  SELECT memberid, member_name, recommendedby, 1 AS level \
  FROM members \
  WHERE member_name = 'Priya'  \
  UNION ALL \
  SELECT m.memberid, m.member_name, m.recommendedby, mc.level + 1 \
  FROM members m \
  JOIN MemberChain mc ON m.memberid = mc.recommendedby \
) \
SELECT * \
FROM MemberChain \
ORDER BY level DESC;\
"

 memberid | member_name | recommendedby | level 
----------+-------------+---------------+-------
 M0001    | Ravi        |               |     2
 M0002    | Priya       | M0001         |     1
(2 rows)



In [28]:
!psql -h localhost -p 5432 -Upostgres -dpraxisdb -c "\
WITH RECURSIVE MemberChain AS ( \
  SELECT memberid, member_name, recommendedby, 1 AS level \
  FROM members \
  WHERE member_name = 'Sunita'  \
  UNION ALL \
  SELECT m.memberid, m.member_name, m.recommendedby, mc.level + 1 \
  FROM members m \
  JOIN MemberChain mc ON m.memberid = mc.recommendedby \
) \
SELECT * \
FROM MemberChain \
ORDER BY level DESC;\
"

 memberid | member_name | recommendedby | level 
----------+-------------+---------------+-------
 M0001    | Ravi        |               |     4
 M0003    | Sanjay      | M0001         |     3
 M0008    | Meena       | M0003         |     2
 M0012    | Sunita      | M0008         |     1
(4 rows)



#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)

In [29]:
from datetime import datetime
import pytz
print('Last tested  ',datetime.now(pytz.timezone('Asia/Kolkata')))

Last tested   2024-03-01 11:37:58.732706+05:30
